**Importing Libraries**

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
import random

In [0]:
usr = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pd-ds-coh-team1/test/users_test.csv')

dvc = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pd-ds-coh-team1/test/devices_test.csv')

tr = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pd-ds-coh-team1/test/transactions_test.csv')

ntf = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pd-ds-coh-team1/test/notifications_test.csv')

In [246]:
tr['created_date']=pd.to_datetime(tr['created_date'])
dict_days_of_transactions = tr.copy()

dict_days_of_transactions['dict_days_of_transactions'] = dict_days_of_transactions['created_date'].dt.date
dict_days_of_transactions = dict_days_of_transactions.groupby(['user_id'])['dict_days_of_transactions'].nunique()
final_dict_days_of_transactions = pd.DataFrame(index=dict_days_of_transactions.axes,data=dict_days_of_transactions.values,columns=['dict_days_of_transactions'])
final_dict_days_of_transactions.head()

,dict_days_of_transactions
user_id,
user_10,297
user_10000,52
user_10001,17
user_10006,18
user_1001,13


In [0]:
usr_dvc = pd.merge(usr,dvc,on='user_id',how='inner')
usr_dvc['age'] = 2020 - usr_dvc['birth_year']

In [0]:
usr_dvc= pd.merge(usr_dvc,final_dict_days_of_transactions,how='left',on='user_id')

In [249]:
usr_dvc.dict_days_of_transactions.isna().sum()

137

In [250]:
def unknown_brand(brand):
  if brand=='Unknown':
    return random.sample(['Android','Apple'],1)[0]
  else:
    return brand

usr_dvc['brand'] = usr_dvc['brand'].apply(unknown_brand)
usr_dvc['brand'].value_counts()

Apple      1967
Android    1919
Name: brand, dtype: int64

In [251]:
devices ={'Android':0 , 'Apple':1}
usr_dvc['brand'] = usr_dvc['brand'].map(devices)

usr_dvc.shape

(3886, 14)

In [0]:
devices ={'Android':0 , 'Apple':1}

usr_dvc['brand'] = usr_dvc['brand'].map(devices)

In [0]:
grouping = {'GB':'western_europe' , 'FR':'western_europe' , 'IE':'western_europe' , 'DE':'western_europe' , 'CH':'western_europe' , 'NL':'western_europe' ,
            'BE':'western_europe' , 'AT':'western_europe' , 'JE':'western_europe' , 'GG':'western_europe' , 'IM':'western_europe',
           'PL':'central_eastern_europe' , 'RO':'central_eastern_europe' , 'CZ':'central_eastern_europe' , 'HU':'central_eastern_europe' , 'BG':'central_eastern_europe' ,
           'SI':'central_eastern_europe' , 'SK':'central_eastern_europe' , 'HR':'central_eastern_europe' , 'LU':'central_eastern_europe' , 'LI':'central_eastern_europe' ,
           'ES' : 'southern_europe', 'PT': 'southern_europe', 'MT': 'southern_europe', 'IT': 'southern_europe', 'GR': 'southern_europe', 'CY': 'southern_europe', 
           'GI': 'southern_europe','LT' : 'northern_europe', 'LV': 'northern_europe', 'SE': 'northern_europe', 'DK': 'northern_europe', 
           'NO': 'northern_europe', 'FI' : 'northern_europe','EE': 'northern_europe','IS': 'northern_europe', 'GP':'not_europe' ,
           'RE':'not_europe','AU':'not_europe' , 'MQ':'not_europe','GF':'not_europe' }

usr_dvc['country'] = usr_dvc['country'].replace(grouping)

In [254]:
usr_dvc['country'].value_counts()

western_europe            2208
central_eastern_europe     858
southern_europe            587
northern_europe            228
not_europe                   5
Name: country, dtype: int64

In [0]:
country_label = usr_dvc['country'].value_counts().index

In [0]:
for i in country_label:
  usr_dvc[i] = np.where(usr_dvc['country']==i,1,0)

In [0]:
tr = tr[(tr['transactions_state'] == 'COMPLETED') & (tr['amount_usd'] > 0)]

In [258]:
tr['transactions_type'].value_counts()

CARD_PAYMENT    258042
TRANSFER         96661
TOPUP            61128
EXCHANGE         34338
ATM              16306
CARD_REFUND       2751
TAX                552
REFUND             300
Name: transactions_type, dtype: int64

In [259]:
types = {'CARD_PAYMENT':'CARD_PAYMENT','TRANSFER':'TRANSFER','TOPUP':'TOPUP','EXCHANGE':'EXCHANGE',
         'ATM':'ATM','REFUND':'CARD_REFUND','CARD_REFUND':'CARD_REFUND',
         'TAX':'FEES'}

tr['transactions_type'] = tr['transactions_type'].map(types)
tr['transactions_type'].value_counts()

CARD_PAYMENT    258042
TRANSFER         96661
TOPUP            61128
EXCHANGE         34338
ATM              16306
CARD_REFUND       3051
FEES               552
Name: transactions_type, dtype: int64

In [0]:
tr_amount_usd = tr.groupby(['user_id'], as_index=True)['amount_usd'].agg(['sum','count','mean'])
tr_amount_usd.rename(columns={'sum':'total_amount', 'count':'number_transactions','mean':'avg_amount'}, inplace=True)

In [261]:
tr_amount_usd.head(1)

,total_amount,number_transactions,avg_amount
user_id,,,
user_10,24493.07,843,29.05465


In [0]:
type_encode = pd.concat([pd.get_dummies(tr['transactions_type']),tr],axis=1)

In [263]:
tr_type = type_encode.groupby(['user_id'], as_index=True)['ATM', 'CARD_PAYMENT', 'CARD_REFUND', 'EXCHANGE', 'FEES', 'TOPUP', 'TRANSFER'].agg('sum').astype(int) 
tr_type.head(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,FEES,TOPUP,TRANSFER
user_id,,,,,,,
user_10,28,366,4,0,0,115,330
user_10000,1,63,2,18,0,22,7


In [0]:
usr_dvc = pd.merge(usr_dvc,tr_amount_usd,how='left',on='user_id')

In [0]:
usr_dvc = pd.merge(usr_dvc,tr_type,how='left',on='user_id')

In [266]:
def currency(currency):
  if currency == 'EUR':
    return 'EUR'
  elif currency == 'GBP':
    return 'GBP'
  elif currency == 'PLN':
    return 'PLN'
  elif currency == 'RON':
    return 'RON'
  elif currency == 'USD':
    return 'USD'
  else:
    return 'OTHER'

tr['transactions_currency'] = tr['transactions_currency'].map(currency)
tr['transactions_currency'].value_counts()

EUR      191163
GBP      159417
OTHER     39731
PLN       36220
RON       28434
USD       15113
Name: transactions_currency, dtype: int64

In [267]:
curr_encode = pd.concat([pd.get_dummies(tr['transactions_currency']),tr],axis=1)
count_curr = curr_encode.groupby(['user_id'])['EUR','GBP','PLN','OTHER','RON','USD'].agg('sum')
usr_dvc = pd.merge(usr_dvc,count_curr,how='left',on='user_id')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [268]:
usr_dvc.head(2)

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand,age,dict_days_of_transactions,western_europe,central_eastern_europe,southern_europe,northern_europe,not_europe,total_amount,number_transactions,avg_amount,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,FEES,TOPUP,TRANSFER,EUR,GBP,PLN,OTHER,RON,USD
0,user_2,1987,central_eastern_europe,Poznań,2018-01-18 19:17:31.229096,0,0.0,0.0,21,0,0,NaN,33,45.0,0,1,0,0,0,1036.54,81.0,12.796790,0.0,31.0,2.0,21.0,0.0,25.0,2.0,4.0,10.0,52.0,0.0,0.0,15.0
1,user_8,1981,western_europe,Fife,2018-01-03 07:28:59.063740,1,NaN,NaN,0,0,0,NaN,39,21.0,1,0,0,0,0,338.09,15.0,22.539333,0.0,5.0,0.0,5.0,0.0,5.0,0.0,0.0,12.0,0.0,3.0,0.0,0.0


In [0]:
ntf = ntf[ntf['status']=='SENT']

In [270]:
reason_encode = pd.concat([pd.get_dummies(ntf['reason'],),ntf],axis=1)
reason_encode.head(2)

,BLACK_FRIDAY,BLUE_TUESDAY,ENGAGEMENT_SPLIT_BILL_RESTAURANT,INVEST_IN_GOLD,JOINING_ANNIVERSARY,LOST_CARD_ORDER,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,METAL_RESERVE_PLAN,NO_INITIAL_CARD_ORDER,NO_INITIAL_CARD_USE,ONBOARDING_TIPS_ACTIVATED_USERS,PROMO,PROMO_CARD_ORDER,REENGAGEMENT_ACTIVE_FUNDS,WELCOME_BACK,reason,channel,status,user_id,created_date
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1408,2018-12-11 02:04:46.284683
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1295,2018-12-15 04:12:57.003255


In [271]:
reason_encode.shape

(17457, 20)

In [272]:
count_reason = reason_encode.groupby(['user_id'], as_index=True)['REENGAGEMENT_ACTIVE_FUNDS', 'PROMO', 'NO_INITIAL_CARD_ORDER'
,'INVEST_IN_GOLD', 'NO_INITIAL_CARD_USE', 'PROMO_CARD_ORDER',  'LOST_CARD_ORDER', 'JOINING_ANNIVERSARY', 'ONBOARDING_TIPS_ACTIVATED_USERS', 'BLACK_FRIDAY'
,'MADE_MONEY_REQUEST_NOT_SPLIT_BILL', 'ENGAGEMENT_SPLIT_BILL_RESTAURANT'
, 'WELCOME_BACK', 'METAL_RESERVE_PLAN'
,'BLUE_TUESDAY', ].agg('sum').astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


In [273]:
usr_dvc = pd.merge(usr_dvc,count_reason,how='left',on='user_id')
usr_dvc.head(2)

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand,age,dict_days_of_transactions,western_europe,central_eastern_europe,southern_europe,northern_europe,not_europe,total_amount,number_transactions,avg_amount,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,FEES,TOPUP,TRANSFER,EUR,GBP,PLN,OTHER,RON,USD,REENGAGEMENT_ACTIVE_FUNDS,PROMO,NO_INITIAL_CARD_ORDER,INVEST_IN_GOLD,NO_INITIAL_CARD_USE,PROMO_CARD_ORDER,LOST_CARD_ORDER,JOINING_ANNIVERSARY,ONBOARDING_TIPS_ACTIVATED_USERS,BLACK_FRIDAY,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,ENGAGEMENT_SPLIT_BILL_RESTAURANT,WELCOME_BACK,METAL_RESERVE_PLAN,BLUE_TUESDAY
0,user_2,1987,central_eastern_europe,Poznań,2018-01-18 19:17:31.229096,0,0.0,0.0,21,0,0,NaN,33,45.0,0,1,0,0,0,1036.54,81.0,12.796790,0.0,31.0,2.0,21.0,0.0,25.0,2.0,4.0,10.0,52.0,0.0,0.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,user_8,1981,western_europe,Fife,2018-01-03 07:28:59.063740,1,NaN,NaN,0,0,0,NaN,39,21.0,1,0,0,0,0,338.09,15.0,22.539333,0.0,5.0,0.0,5.0,0.0,5.0,0.0,0.0,12.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
mask = usr_dvc['attributes_notifications_marketing_push'].isna()
ind =usr_dvc['attributes_notifications_marketing_push'].loc[mask].sample(frac=0.94).index
usr_dvc.loc[ind,'attributes_notifications_marketing_push'] = 1
usr_dvc['attributes_notifications_marketing_push']= usr_dvc['attributes_notifications_marketing_push'].fillna(0)

In [0]:

mask = usr_dvc['attributes_notifications_marketing_email'].isna()
ind = usr_dvc['attributes_notifications_marketing_email'].loc[mask].sample(frac=0.89).index
usr_dvc.loc[ind,'attributes_notifications_marketing_email'] = 1
usr_dvc['attributes_notifications_marketing_email']= usr_dvc['attributes_notifications_marketing_email'].fillna(0)

In [276]:
usr_dvc.describe()

,birth_year,user_settings_crypto_unlocked,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand,age,dict_days_of_transactions,western_europe,central_eastern_europe,southern_europe,northern_europe,not_europe,total_amount,number_transactions,avg_amount,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,FEES,TOPUP,TRANSFER,EUR,GBP,PLN,OTHER,RON,USD,REENGAGEMENT_ACTIVE_FUNDS,PROMO,NO_INITIAL_CARD_ORDER,INVEST_IN_GOLD,NO_INITIAL_CARD_USE,PROMO_CARD_ORDER,LOST_CARD_ORDER,JOINING_ANNIVERSARY,ONBOARDING_TIPS_ACTIVATED_USERS,BLACK_FRIDAY,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,ENGAGEMENT_SPLIT_BILL_RESTAURANT,WELCOME_BACK,METAL_RESERVE_PLAN,BLUE_TUESDAY
count,3886.000000,3886.000000,3886.000000,3886.000000,3886.000000,3886.0,3886.0,0.0,3886.000000,3749.000000,3886.000000,3886.000000,3886.000000,3886.000000,3886.000000,3.701000e+03,3701.00000,3701.000000,3701.000000,3701.000000,3701.000000,3701.000000,3701.000000,3701.000000,3701.000000,3701.000000,3701.000000,3701.000000,3701.000000,3701.000000,3701.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000,3673.000000
mean,1983.576686,0.175502,0.943129,0.895265,17.941585,0.0,0.0,NaN,36.423314,47.449987,0.568194,0.220793,0.151055,0.058672,0.001287,8.509095e+03,127.01378,83.760437,4.405836,69.722237,0.824372,9.278033,0.149149,16.516617,26.117536,51.651716,43.074034,9.786544,10.735207,7.682788,4.083491,1.523822,0.908249,0.430711,0.381160,0.293221,0.240675,0.231963,0.182684,0.174244,0.100191,0.084672,0.090389,0.038116,0.043289,0.029404
std,11.368799,0.380445,0.231625,0.306251,42.080140,0.0,0.0,NaN,11.368799,57.038711,0.495392,0.414834,0.358149,0.235040,0.035852,3.519524e+04,220.44638,275.793034,11.194840,122.187055,4.663250,35.405355,1.473887,23.731199,80.587102,125.478925,141.475714,47.884668,61.252897,70.051312,34.080355,1.818431,0.929732,0.660225,0.485738,0.654131,0.641104,0.580260,0.386460,0.800749,0.300295,0.278431,0.286778,0.191502,0.408954,0.194438
min,1934.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,19.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e-02,1.00000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1978.000000,0.000000,1.000000,1.000000,0.000000,0.0,0.0,NaN,28.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.667500e+02,14.00000,17.054853,0.000000,3.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1986.000000,0.000000,1.000000,1.000000,4.000000,0.0,0.0,NaN,34.000000,26.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.235680e+03,50.00000,35.574069,1.000000,25.000000,0.000000,0.000000,0.000000,8.000000,3.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1992.000000,0.000000,1.000000,1.000000,19.000000,0.0,0.0,NaN,42.000000,66.000000,1.000000,0.000000,0.000000,0.000000,0.000000,7.168530e+03,140.00000,68.525430,5.000000,79.000000,1.000000,6.000000,0.000000,20.000000,15.000000,42.000000,20.000000,0.000000,0.000000,0.000000,0.000000,3.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2001.000000,1.000000,1.000000,1.000000,732.000000,0.0,0.0,NaN,86.000000,378.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.470283e+06,3878.00000,5881.714659,323.000000,1335.000000,221.000000,933.000000,44.000000,254.000000,1

In [0]:
usr_dvc = usr_dvc.fillna(0)

In [278]:
usr_dvc.columns

Index(['user_id', 'birth_year', 'country', 'city', 'created_date',
       'user_settings_crypto_unlocked',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts',
       'num_referrals', 'num_successful_referrals', 'brand', 'age',
       'dict_days_of_transactions', 'western_europe', 'central_eastern_europe',
       'southern_europe', 'northern_europe', 'not_europe', 'total_amount',
       'number_transactions', 'avg_amount', 'ATM', 'CARD_PAYMENT',
       'CARD_REFUND', 'EXCHANGE', 'FEES', 'TOPUP', 'TRANSFER', 'EUR', 'GBP',
       'PLN', 'OTHER', 'RON', 'USD', 'REENGAGEMENT_ACTIVE_FUNDS', 'PROMO',
       'NO_INITIAL_CARD_ORDER', 'INVEST_IN_GOLD', 'NO_INITIAL_CARD_USE',
       'PROMO_CARD_ORDER', 'LOST_CARD_ORDER', 'JOINING_ANNIVERSARY',
       'ONBOARDING_TIPS_ACTIVATED_USERS', 'BLACK_FRIDAY',
       'MADE_MONEY_REQUEST_NOT_SPLIT_BILL', 'ENGAGEMENT_SPLIT_BILL_RESTAURANT',
       'WELCOME_BACK', 'METAL_RESERVE_PLAN', 'BLUE_TUE

In [0]:
usr_dvc.to_csv('test_set.txt',index=False)

**Χ_test , X_train , Y_train**

In [0]:
X_test = usr_dvc.drop(['user_id','birth_year', 'country', 'city', 'created_date', 'num_referrals','num_successful_referrals',],axis=1)

In [281]:
X_test.shape

(3886, 43)

In [282]:
X_test.columns

Index(['user_settings_crypto_unlocked',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts', 'brand',
       'age', 'dict_days_of_transactions', 'western_europe',
       'central_eastern_europe', 'southern_europe', 'northern_europe',
       'not_europe', 'total_amount', 'number_transactions', 'avg_amount',
       'ATM', 'CARD_PAYMENT', 'CARD_REFUND', 'EXCHANGE', 'FEES', 'TOPUP',
       'TRANSFER', 'EUR', 'GBP', 'PLN', 'OTHER', 'RON', 'USD',
       'REENGAGEMENT_ACTIVE_FUNDS', 'PROMO', 'NO_INITIAL_CARD_ORDER',
       'INVEST_IN_GOLD', 'NO_INITIAL_CARD_USE', 'PROMO_CARD_ORDER',
       'LOST_CARD_ORDER', 'JOINING_ANNIVERSARY',
       'ONBOARDING_TIPS_ACTIVATED_USERS', 'BLACK_FRIDAY',
       'MADE_MONEY_REQUEST_NOT_SPLIT_BILL', 'ENGAGEMENT_SPLIT_BILL_RESTAURANT',
       'WELCOME_BACK', 'METAL_RESERVE_PLAN', 'BLUE_TUESDAY'],
      dtype='object')

In [0]:
data = pd.read_csv ('/content/train_set_final (1).txt')

In [284]:
data.columns

Index(['user_id', 'birth_year', 'country', 'city', 'created_date',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts',
       'num_referrals', 'num_successful_referrals', 'brand', 'age',
       'dict_days_of_transactions', 'western_europe', 'central_eastern_europe',
       'southern_europe', 'northern_europe', 'not_europe', 'total_amount',
       'number_transactions', 'avg_amount', 'ATM', 'CARD_PAYMENT',
       'CARD_REFUND', 'EXCHANGE', 'CASHBACK', 'FEES', 'TOPUP', 'TRANSFER',
       'EUR', 'GBP', 'PLN', 'OTHER', 'RON', 'USD', 'REENGAGEMENT_ACTIVE_FUNDS',
       'PROMO', 'NO_INITIAL_CARD_ORDER', 'INVEST_IN_GOLD',
       'NO_INITIAL_CARD_USE', 'PROMO_CARD_ORDER', 'LOST_CARD_ORDER',
       'JOINING_ANNIVERSARY', 'ONBOARDING_TIPS_ACTIVATED_USERS',
       'BLACK_FRIDAY', 'MADE_MONEY_REQUEST_NOT_SPLIT_BILL',
       'ENGAGEMENT_SPLIT_BILL_RESTAURANT', 'SILVER_ENGAGEMENT_INACTIVE

In [0]:
X_train = data.drop(['user_id','birth_year','country','city','created_date','plan','num_referrals','num_successful_referrals','CASHBACK','SILVER_ENGAGEMENT_INACTIVE_CARD','SILVER_ENGAGEMENT_FEES_SAVED' ],axis=1)

In [286]:
X_train.shape

(15544, 43)

In [0]:
y_train = data['plan']

In [288]:
X_train.shape, y_train.shape ,X_test.shape

((15544, 43), (15544,), (3886, 43))

**Gradient Boosting Classifier**

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

In [290]:
sc = StandardScaler()
var = VarianceThreshold()
sampler = SMOTE()
mdl= GradientBoostingClassifier()




pipes = Pipeline([('scaler', sc),
                  ('selector', var),
                  ('sampler', sampler),
                  ('model', mdl)])

grid= {'selector__threshold': [0.4],
         'model__learning_rate': [0.1],
         'model__n_estimators':[250]
         
         }

clf= GridSearchCV(pipes,grid,cv=5)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('selector',
                                        VarianceThreshold(threshold=0.0)),
                                       ('sampler',
                                        SMOTE(k_neighbors=5, kind='deprecated',
                                              m_neighbors='deprecated',
                                              n_jobs=1, out_step='deprecated',
                                              random_state=None, ratio=None,
                                              sampling_strategy='auto',
                                              svm_est...
                                             

**Predictions**

In [0]:
predicts =pd.DataFrame(clf.predict(X_test))

In [292]:
predicts[0].value_counts()

0    3865
1      21
Name: 0, dtype: int64

In [0]:
usr_dvc['predicts'] = predicts

In [0]:
dataframe = usr_dvc['user_id']

In [0]:
dataframe=pd.DataFrame(dataframe)

In [0]:
dataframe['predicts']=usr_dvc['predicts']

In [0]:
dataframe.to_csv('test_preds_final.csv',index=False)